In [ ]:
import re
import os
import numpy as np
import scipy.interpolate as interp
from scipy.ndimage import map_coordinates

from robotcar.python.camera_model import CameraModel
from robotcar.python.transform import build_se3_transform
from robotcar.python.image import load_image
from robotcar.python.interpolate_poses import interpolate_vo_poses, interpolate_ins_poses

from monodepth.utils.evaluation_utils import compute_errors

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import shutil

import cv2

In [25]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

In [34]:
# Width and height of ground truth
height = 960
width = 1280

def convert_disps_to_depths(pred_disp, height, width):

    pred_disp = width * cv2.resize(pred_disp, (width, height), interpolation=cv2.INTER_LINEAR)
    pred_depths = 983.044006 * 0.24 / pred_disp
    return pred_depths

### Directories

In [29]:
image_dir = '/media/psf/Data/Downloads/2014-12-05-11-09-10/2014-12-05-11-09-10_stereo_left_02/2014-12-05-11-09-10/stereo/left/'
extrinsics_dir = 'robotcar/extrinsics/'
models_dir = 'robotcar/models/'
lidar_dir = '/media/psf/Data/Downloads/2014-12-05-11-09-10/2014-12-05-11-09-10_ldmrs/ldmrs/'

lidar = 'ldmrs'

vo_poses_file = '/media/psf/Data/Downloads/2014-12-05-11-09-10/2014-12-05-11-09-10_vo/vo/vo.csv'
ins_poses_file = '/media/psf/Data/Downloads/2014-12-05-11-09-10/2014-12-05-11-09-10_gps/gps/ins.csv'

test_image_dir = 'test_images'

pred_disparities_file = 'test_images/disparities.npy'

#### Which image(s)

In [7]:
img_index_start = 10050
img_index_stop = 12000
img_index_step = 100

# micro!!s history for pointcloud generation
time_minus = 10000000
time_plus = 0

### Camera and LIDAR Extrinsics

In [8]:
model = CameraModel(models_dir, image_dir)

extrinsics_path = os.path.join(extrinsics_dir, model.camera + '.txt')
with open(extrinsics_path) as extrinsics_file:
    extrinsics = [float(x) for x in next(extrinsics_file).split(' ')]

G_camera_vehicle = build_se3_transform(extrinsics)

with open(os.path.join(extrinsics_dir, lidar + '.txt')) as extrinsics_file:
    extrinsics = next(extrinsics_file)
G_posesource_laser = build_se3_transform([float(x) for x in extrinsics.split(' ')])

# Preprocess Images and Save to test folder

In [18]:
# Create/clear test_images folder and create filenames file
if os.path.exists(test_image_dir):
    shutil.rmtree(test_image_dir) 
os.makedirs(test_image_dir)

with open(test_image_dir + '/test_images.txt', "a+") as filenames_file:
    
    for img_index in range(img_index_start, img_index_stop, img_index_step):
        timestamps_path = os.path.join(image_dir, os.pardir, model.camera + '.timestamps')
        if not os.path.isfile(timestamps_path):
            timestamps_path = os.path.join(image_dir, os.pardir, os.pardir, model.camera + '.timestamps')

        with open(timestamps_path) as timestamps_file:
            for i, line in enumerate(timestamps_file):
                if i == img_index:
                    timestamp_img = int(line.split(' ')[0])

        image_path = os.path.join(image_dir, str(timestamp_img) + '.png')
        image = load_image(image_path, model)
        img = Image.fromarray(image)
        img.save('test_images/' + str(timestamp_img) + '.png')

        # Append to filenames file
        if img_index == img_index_start:
            filenames_file.write(str(timestamp_img) + '.png')
        else:
            filenames_file.write('\n' + str(timestamp_img) + '.png')
filenames_file.close()

# [Monodept] Get Predictions from Model


In [22]:
!python monodepth/monodepth_main.py --mode test --data_path test_images/ --filenames_file test_images/test_images.txt --log_directory log/ --checkpoint_path monodepth/model_kitti/model_kitti.ckpt --output_directory test_images/


now testing 20 files
step 0 from 20
step 1 from 20
step 2 from 20
step 3 from 20
step 4 from 20
step 5 from 20
step 6 from 20
step 7 from 20
step 8 from 20
step 9 from 20
step 10 from 20
step 11 from 20
step 12 from 20
step 13 from 20
step 14 from 20
step 15 from 20
step 16 from 20
step 17 from 20
step 18 from 20
step 19 from 20
done.
writing disparities.
done.


# [Monodepth] Convert predicted disparities to depth

In [31]:
pred_disparities = np.load(pred_disparities_file)

print(pred_disparities.shape)

(20, 256, 512)


# Process Pointclouds and save ground truth
## TODO save ground truth to file

In [27]:
timestamps_path = os.path.join(lidar_dir, os.pardir, lidar + '.timestamps')

# Loop trough image indices
for img_index in range(img_index_start, img_index_stop, img_index_step):
    
    # Get current image timestamp
    timestamps_path = os.path.join(image_dir, os.pardir, model.camera + '.timestamps')
    if not os.path.isfile(timestamps_path):
        timestamps_path = os.path.join(image_dir, os.pardir, os.pardir, model.camera + '.timestamps')

    with open(timestamps_path) as timestamps_file:
        for i, line in enumerate(timestamps_file):
            if i == img_index:
                timestamp_img = int(line.split(' ')[0])

    timestamps = []
    with open(timestamps_path) as timestamps_file:
        for line in timestamps_file:
            timestamp = int(line.split(' ')[0])
            if (timestamp_img-time_minus) <= timestamp <= (timestamp_img+time_plus):
                timestamps.append(timestamp)

    print(len(timestamps), "LIDAR Scans found")

    if len(timestamps) == 0:
        raise ValueError("No LIDAR data in the given time bracket.")

    # VO Pointcloud
    
    # sensor is VO, which is located at the main vehicle frame
    poses = interpolate_vo_poses(vo_poses_file, timestamps, timestamp_img)

    pointcloud_vo = np.array([[0], [0], [0], [0]])

    for i in range(0, len(poses)):
        scan_path = os.path.join(lidar_dir, str(timestamps[i]) + '.bin')
        if not os.path.isfile(scan_path):
            continue

        scan_file = open(scan_path)
        scan = np.fromfile(scan_file, np.double)
        scan_file.close()
        scan = scan.reshape((len(scan) // 3, 3)).transpose()
        scan = np.dot(np.dot(poses[i], G_posesource_laser), np.vstack([scan, np.ones((1, scan.shape[1]))]))
        pointcloud_vo = np.hstack([pointcloud_vo, scan])

    pointcloud_vo = pointcloud_vo[:, 1:]
    pointcloud_vo = np.dot(G_camera_vehicle, pointcloud_vo)
    print("VO Pointcloud build")
    
    # INS Pointcloud
    
    with open(os.path.join(extrinsics_dir, 'ins.txt')) as extrinsics_file:
        extrinsics = next(extrinsics_file)
        G_posesource_laser = np.linalg.solve(build_se3_transform([float(x) for x in extrinsics.split(' ')]),
                                             G_posesource_laser)
        G_camera_posesource = G_camera_vehicle * build_se3_transform([float(x) for x in extrinsics.split(' ')])

    poses = interpolate_ins_poses(ins_poses_file, timestamps, timestamp_img)

    pointcloud_ins = np.array([[0], [0], [0], [0]])

    for i in range(0, len(poses)):
        scan_path = os.path.join(lidar_dir, str(timestamps[i]) + '.bin')
        if not os.path.isfile(scan_path):
            continue

        scan_file = open(scan_path)
        scan = np.fromfile(scan_file, np.double)
        scan_file.close()
        scan = scan.reshape((len(scan) // 3, 3)).transpose()
        scan = np.dot(np.dot(poses[i], G_posesource_laser), np.vstack([scan, np.ones((1, scan.shape[1]))]))
        pointcloud_ins = np.hstack([pointcloud_ins, scan])

    pointcloud_ins = pointcloud_ins[:, 1:]
    pointcloud_ins = np.dot(G_camera_posesource, pointcloud_ins)
    print("INS Pointcloud build")

    # Sparse Local Pointcloud
    
    with open(os.path.join(extrinsics_dir, 'ins.txt')) as extrinsics_file:
        extrinsics = next(extrinsics_file)
        G_posesource_laser = np.linalg.solve(build_se3_transform([float(x) for x in extrinsics.split(' ')]),
                                             G_posesource_laser)
        G_camera_posesource = G_camera_vehicle * build_se3_transform([float(x) for x in extrinsics.split(' ')])

    poses = interpolate_ins_poses(ins_poses_file, timestamps, timestamp_img)

    pointcloud_ins = np.array([[0], [0], [0], [0]])

    for i in range(0, len(poses)):
        scan_path = os.path.join(lidar_dir, str(timestamps[i]) + '.bin')
        if not os.path.isfile(scan_path):
            continue

        scan_file = open(scan_path)
        scan = np.fromfile(scan_file, np.double)
        scan_file.close()
        scan = scan.reshape((len(scan) // 3, 3)).transpose()
        scan = np.dot(np.dot(poses[i], G_posesource_laser), np.vstack([scan, np.ones((1, scan.shape[1]))]))
        pointcloud_ins = np.hstack([pointcloud_ins, scan])

    pointcloud_ins = pointcloud_ins[:, 1:]
    pointcloud_ins = np.dot(G_camera_posesource, pointcloud_ins)

    print("Sparse Pointcloud build")

(161, 'LIDAR Scans found')
VO Pointcloud build


robotcar/python/interpolate_poses.py:141: RuntimeWarning: divide by zero encountered in divide
  (pose_timestamps[upper_indices] - pose_timestamps[lower_indices])


INS Pointcloud build
Sparse Pointcloud build
(161, 'LIDAR Scans found')
VO Pointcloud build
INS Pointcloud build
Sparse Pointcloud build
(161, 'LIDAR Scans found')
VO Pointcloud build


KeyboardInterrupt: 

# Evaluate Test Result with Ground Truth
## TODO implement for multiple images

In [ ]:
pred_disparities = np.load('test_images/test_disp.npy')

pred_depth = convert_disps_to_depths(pred_disparities, 960, 1280)

## Show Error Magnitude

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,20))

# Sparse Map
ax = plt.subplot(3, 2, 1)
ax.set_title("Local PC - Pred Error (m) [Pred - GT]", fontsize=16)
uv, depth = model.project(local_pointcloud, image.shape)

uv = np.array(uv)
depth = np.array(depth)
mask = ((uv[0,:] < 1279) & (uv[1,:] < 959) & (depth < 80))
depth = depth[mask]
uv = uv[:, mask]

gt_img = np.zeros((960, 1280))
coords = uv.transpose().astype(int)
gt_img[coords[:,1],coords[:,0]] = depth
compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
abs_rel, sq_rel, rms, log_rms, a1, a2, a3 = compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
print("Errors for Local LIDAR:")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))

pixels = uv.transpose()
# 1200 x 960
pred_depth_pixels = []
pred_depth_error = []
i = 0
for pixel in pixels:
    # Sparse
    pred_depth_pixels.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))])
    pred_depth_error.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))] - depth[i])
    i += 1

# Limit coordinates to image size
pred_depth_pixels = np.array(pred_depth_pixels)
pred_depth_error = np.array(pred_depth_error)
mask = (pred_depth_pixels < 80)
depth = depth[mask]
uv = uv[:, mask]
pred_depth_error = pred_depth_error[mask]
pred_depth_pixels = pred_depth_pixels[mask]
    
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=pred_depth_error,
                 edgecolors='none', norm=MidpointNormalize(midpoint=0.) , cmap='RdYlGn')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 2)
ax.set_title("Local PC - Pred Error (Rel. Mag.) [Pred - GT]", fontsize=16)
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=np.abs(pred_depth_error/depth),
                 edgecolors='none', cmap='Reds')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 3)
ax.set_title("VO PC - Pred Error (m) [Pred - GT]", fontsize=16)
uv, depth = model.project(pointcloud_vo, image.shape)

# Limit coordinates to image size
uv = np.array(uv)
depth = np.array(depth)
mask = ((uv[0,:] < 1279) & (uv[1,:] < 959) & (depth < 80))
depth = depth[mask]
uv = uv[:, mask]

gt_img = np.zeros((960, 1280))
coords = uv.transpose().astype(int)
gt_img[coords[:,1],coords[:,0]] = depth
compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
abs_rel, sq_rel, rms, log_rms, a1, a2, a3 = compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
print("Errors for INS LIDAR:")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))

pixels = uv.transpose()
# 1200 x 960
pred_depth_pixels = []
pred_depth_error = []
i = 0
for pixel in pixels:
    # Sparse
    pred_depth_pixels.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))])
    pred_depth_error.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))] - depth[i])
    i += 1

# Limit coordinates to image size
pred_depth_pixels = np.array(pred_depth_pixels)
pred_depth_error = np.array(pred_depth_error)
mask = (pred_depth_pixels < 80)
depth = depth[mask]
uv = uv[:, mask]
pred_depth_error = pred_depth_error[mask]
pred_depth_pixels = pred_depth_pixels[mask]
    
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=pred_depth_error, norm=MidpointNormalize(midpoint=0.), cmap='RdYlGn',
                 edgecolors='none')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 4)
ax.set_title("VO PC - Pred Error (Rel. Mag.) [Pred - GT]", fontsize=16)
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=np.abs(pred_depth_error/depth),
                 edgecolors='none', cmap='Reds')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 5)
ax.set_title("INS PC - Pred Error (m) [Pred - GT]", fontsize=16)
uv, depth = model.project(pointcloud_ins, image.shape)

# Limit coordinates to image size
uv = np.array(uv)
depth = np.array(depth)
mask = ((uv[0,:] < 1279) & (uv[1,:] < 959) & (depth < 80))
depth = depth[mask]
uv = uv[:, mask]

gt_img = np.zeros((960, 1280))
coords = uv.transpose().astype(int)
gt_img[coords[:,1],coords[:,0]] = depth
compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
abs_rel, sq_rel, rms, log_rms, a1, a2, a3 = compute_errors(gt_img[gt_img != 0], pred_depth[gt_img != 0])
print("Errors for VO LIDAR:")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))

pixels = uv.transpose()
# 1200 x 960
pred_depth_pixels = []
pred_depth_error = []
i = 0
for pixel in pixels:
    # Sparse
    pred_depth_pixels.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))])
    pred_depth_error.append(pred_depth[int(round(pixel[1]))][int(round(pixel[0]))] - depth[i])
    i += 1

# Limit coordinates to image size
pred_depth_pixels = np.array(pred_depth_pixels)
pred_depth_error = np.array(pred_depth_error)
mask = (pred_depth_pixels < 80)
depth = depth[mask]
uv = uv[:, mask]
pred_depth_error = pred_depth_error[mask]
pred_depth_pixels = pred_depth_pixels[mask]
    
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=pred_depth_error, edgecolors='none', norm=MidpointNormalize(midpoint=0.), cmap='RdYlGn')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

ax = plt.subplot(3, 2, 6)
ax.set_title("INS PC - Pred Error (Rel. Mag.) [Pred - GT]", fontsize=16)
plt.imshow(image)
plt.hold(True)
sc = plt.scatter(np.ravel(uv[0, :]), np.ravel(uv[1, :]), s=20, c=np.abs(pred_depth_error/depth),
                 edgecolors='none', cmap='Reds')
plt.colorbar(sc)
plt.xlim(0, image.shape[1])
plt.ylim(image.shape[0], 0)
plt.xticks([])
plt.yticks([])

plt.savefig('result_imgs/' + str(timestamp_img) + 'results.png')